In [1]:
from IPython.display import clear_output
!pip install mediapipe
!pip install bing-image-downloader
!pip install deepface
clear_output()

In [1]:
import os
import json
import time
import cv2
import numpy as np
import mediapipe as mp
import math
import csv 

In [2]:
class FaceDetector():
    def __init__(self, minDetectionCon=0.5):
        self.minDetectionCon = minDetectionCon
        self.mpDraw = mp.solutions.drawing_utils
        self.mpFaceDetection = mp.solutions.face_detection
        self.faceDetection = self.mpFaceDetection.FaceDetection(self.minDetectionCon, model_selection=1)
    def findFace(self, img):
        self.imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.faceDetection.process(self.imgRGB)
        faces = []
        cropped_faces = []
        if self.results.detections:
            for detection in self.results.detections:
                key_points = detection.location_data.relative_keypoints

                # Get the location of the left and right eyes
                left_eye = (
                    int(key_points[self.mpFaceDetection.FaceKeyPoint.LEFT_EYE].x * img.shape[1]),
                    int(key_points[self.mpFaceDetection.FaceKeyPoint.LEFT_EYE].y * img.shape[0])
                )
                right_eye = (
                    int(key_points[self.mpFaceDetection.FaceKeyPoint.RIGHT_EYE].x * img.shape[1]),
                    int(key_points[self.mpFaceDetection.FaceKeyPoint.RIGHT_EYE].y * img.shape[0])
                )
                delta_x = right_eye[0] - left_eye[0]
                delta_y = right_eye[1] - left_eye[1]
                alpha = np.degrees(np.arctan2(delta_y, delta_x))

                box = detection.location_data.relative_bounding_box
                ih, iw, ic = img.shape
                x1 = int(box.xmin * img.shape[1])
                y1 = int(box.ymin * img.shape[0])
                x2 = int((box.xmin + box.width) * iw)
                y2 = int((box.ymin + box.height) * ih)
                faces.append([x1, y1, x2, y2])
                # face_image = img[y1:y2, x1:x2]
                face_image = img[y1-50:y2+50, x1-50:x2+50]
                rotated_image = cv2.warpAffine(face_image,
                                cv2.getRotationMatrix2D((face_image.shape[1] / 2, 
                                face_image.shape[0] / 2), alpha+180, 1.0),
                                (face_image.shape[1], face_image.shape[0]))
                #resize face to train facenet model
                resized_image = cv2.resize(rotated_image, (160, 160))
                # cv2.imshow("Cropped Face", resized_image)

                # Append the cropped face to the list
                cropped_faces.append(resized_image)
        return  faces,cropped_faces

In [3]:
from deepface import DeepFace
def deep_data_extract(img):
    face_data=None
    try:
        embedding = DeepFace.represent(img, model_name='Facenet')
        # print(embedding[0]["embedding"])
        face_data=embedding[0]["embedding"]
    except:
        pass
    return face_data

In [4]:
detector = FaceDetector(minDetectionCon=0.5)

In [10]:
import os
import shutil
if os.path.exists("./download"):
   shutil.rmtree("./download")
if not os.path.exists("./download"):
  os.mkdir("./download")
from bing_image_downloader import downloader
def download_image(query,limit):
  try:
    downloader.download(query, limit=limit, output_dir='./download')
  except:
    pass
celebrity_list=["celebrity","women","men"]
for i in celebrity_list:
  download_image(i,100)    

clear_output()  

In [11]:
import string
import random
def generate_name():
  # Define the length of the file name
  length = 8
  # Define the set of characters to choose from
  chars = string.ascii_letters + string.digits
  # Generate a random file name
  file_name = ''.join(random.choice(chars) for _ in range(length))
  # Print the file name
  return  file_name

  
if os.path.exists("./dataset"):
    shutil.rmtree("./dataset")
#     os.mkdir("./output")
if not os.path.exists("./dataset"):
    os.makedirs("./dataset")

for i in os.listdir("./download"):
  for j in os.listdir(f"./download/{i}"):
    try:
      extenstion=j.split(".")[-1]
      shutil.copy(f"./download/{i}/{j}",f"./dataset/{generate_name()}.{extenstion}")
    except:
      pass    

In [5]:
len(os.listdir("./dataset"))

300

In [9]:
def utils(img):
  # cv2_imshow(img)
  faces,cropped_faces= detector.findFace(img)
  if len(faces) != 0:
    for frame in cropped_faces:
      face_data=deep_data_extract(frame)
      if face_data!=None:
        # cv2_imshow(frame) 
        # print(face_data)
        return face_data
        


In [7]:
import csv
def unknown_csv(face_data):
    with open('unknown.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([face_data, "unknown",0,"None","None"])

In [11]:
import cv2
from google.colab.patches import cv2_imshow
for i in os.listdir("./dataset"):
  img=cv2.imread(f"./dataset/{i}")
  try:
    face_data=utils(img)
    if face_data!=None:
      unknown_csv(face_data)
  except:
    pass
